# Ottenimento prossimi match
Lo script deve permettere di prendere i prossimi 10 match. 
Verrà fatta una predizione su questi match, prima deve essere calcolata la media di ciascun match che è data dai 5 match precedenti.

In [1]:
import pandas as pd

In [2]:
championship = pd.read_csv('files/Serie A/2022-2023 championship_matches.csv', index_col=0)
championship['date'] = pd.to_datetime(championship['date'], format='%Y-%m-%d')

championship.result = championship.result.astype(str)
new_matches = championship[championship.result == 'nan']

## Dati inconsistenti?
Prendo la data del primo match nei match che vengono estratti e se la data di oggi è successiva a quella data, significa che i dati non sono aggiornati, quindi riscarico il dataset della stagione attuale

In [3]:
from datetime import datetime
# Getting the current date and time
dt = datetime.now()

first_date = new_matches.iloc[0].date
first_date = first_date.to_pydatetime()

if dt.date() > first_date.date():
    print("You must download the actual season or the prediction will be inconsistent")    

## Visualizzo i prossimi match

In [4]:
matches = new_matches[:10][['date', 'team1', 'team2']]

In [5]:
from ranking import Ranking
seasons = 4
r = Ranking("Serie A", '2022-2023')
r.read_matches(seasons=seasons)

matches.reset_index(drop=True,inplace=True)

In [6]:
import matplotlib.pyplot as plt
import numpy as np
str = "Scontri diretti nelle 4 stagioni precedenti di campionato"

for i, m in matches.iterrows():
    rankH, rankA = r.get_rank(team_name1=m.team1, team_name2=m.team2)

    matches.at[i, 'rank_h'] = rankH
    matches.at[i, 'rank_a'] = rankA
    
    teams = [m.team1+" wins", m.team2+" wins", "draws"]
    """values = r.get_direct_stats()
    plt.figure(figsize=(15, 3))
    team1_vs_team2_total_matches = np.sum(values)
    plt.plot(team1_vs_team2_total_matches)
    plt.bar(teams, values)
    plt.title(m.team1+' vs '+m.team2+' in the last 4 seasons: {} total matches'.format(team1_vs_team2_total_matches))
    plt.show()
    """
matches.rename(columns={'team1':'home', 'team2':'away'}, inplace=True)

## Ottenimento testo predizioni

In [7]:
from temp.football_predictions import FootballPredictions

fp = FootballPredictions(matches)
fp.add_awayNotation()
fp.get_urls()

100%|██████████| 4/4 [00:01<00:00,  2.67it/s]


In [8]:
fp.get_predictions()
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates()

100%|██████████| 4/4 [00:02<00:00,  1.83it/s]


## Tokenizzazione del testo

In [9]:
from my_tokenizer import MyTokenizer
from classification import Classification
from model import Model
import pandas as pd

In [10]:
mt = MyTokenizer(pd.read_csv("files/cleaned_descriptive_predictions.csv", index_col=0))
mt.feature_normalization()
mt.clean_text()
mt.set_bigram()
#X_train, X_test, y_train, y_test = mt.set_bigram_and_get_sets()

<10x712 sparse matrix of type '<class 'numpy.float64'>'
	with 835 stored elements in Compressed Sparse Row format>

In [18]:
c = Classification(None, mt.set_bigram(), None, None)
import pickle


loaded_model = pickle.load(open('Decision Tree stats.model', 'rb'))
loaded_model.predict(mt.set_bigram())


ValueError: X has 712 features, but DecisionTreeClassifier is expecting 17555 features as input.